$m_1$ masės parašiutininkas su $m_2$ masės įranga iššoka iš lėktuvo, kuris skrenda aukštyje $h_0$. Po $t_g$ laisvo kritimo parašiutas išskleidžiamas. Oro pasipriešinimo koeficientas laisvo kritimo metu lygus $k_1$, o išskleidus parašiutą - $k_2$. Tariama, kad paliekant lėktuvą parašiutininko greitis lygus 0 m/s, o oro pasipriešinimas proporcingas parašiutininko greičio kvadratui. Raskite, kaip kinta parašiutininko greitis nuo 0 s iki nusileidimo. Kada ir kokiu greičiu parašiutininkas pasiekia žemę? Kokiame aukštyje išskleidžiamas parašiutas?
| Varianto numeris | $m_1$, kg | $m_2$, kg | $h_0$, m | $t_g$, s | $k_1$, kg/m | $k_2$, kg/m |
|---|---|---|---|---|---|---|
| 11 | 100 | 15 | 3000 | 40 | 0,5 | 10 |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate

# Constants
m1 = 100 #kg mass of skydiver
m2 = 15 #kg mass of skydiver's equipment
m = m1 + m2 #kg total mass of skydiver
k1 = 0.5 #kg/m drag coefficient of skydiver w/o parachute
k2 = 10 #kg/m drag coefficient of skydiver w/ parachute
v0 = 0 #m/s initial velocity
tg = 40 #s time of opening parachute
h0 = 3000 #m initial height
g = 9.81 #m/s^2 gravitational acceleration

totalTime = 175 #s total time of simulation
iterations = 1000 #number of iterations

def function(time, data):
    """
    Calculate the velocity and acceleration at a given time.

    Parameters:
    time (float): The current time.
    data (array): The current height and velocity.

    Returns:
    new_data (array): The new height and velocity.
    """
    vCurr = data[1]
    new_data = np.zeros(2, dtype=float)

    new_data[0] = vCurr
    if time < tg:
        new_data[1] = - ((m * g - k1 * vCurr**2) / m)
    else:
        new_data[1] = - ((m * g - k2 * vCurr**2) / m)

    return new_data

def isSkydiverOnGround(data):
    """
    Check if the skydiver has reached the ground.

    Parameters:
    data (array): The current height and velocity.

    Returns:
    data (array): The updated height and velocity.
    """
    hCurr = data[0]
    if hCurr <= 0:
        data[0] = 0
        data[1] = 0
    return data


def EulerStep(data, tCurr, dt):
    """
    Perform one step of the Euler method.

    Parameters:
    data (array): The current height and velocity.
    tCurr (float): The current time.
    dt (float): The time step.

    Returns:
    new_data (array): The updated height and velocity.
    """
    new_data = data + function(tCurr, data) * dt

    # Check if skydiver is on the ground
    new_data = isSkydiverOnGround(new_data)

    return new_data

def RK4(data, tCurr, dt):
    """
    Perform one step of the RK4 method.

    Parameters:
    data (array): The current height and velocity.
    tCurr (float): The current time.
    dt (float): The time step.

    Returns:
    new_data (array): The updated height and velocity.
    """
    k0 = function(tCurr, data)
    k1 = data + dt/2 * function(tCurr, data)
    k2 = data + dt/2 * function(tCurr + dt/2, k1)
    k3 = data + dt * function(tCurr + dt/2, k2)

    new_data = data + dt/6 * (
        k0
        + 2 * function(tCurr + dt/2, k1)
        + 2 * function(tCurr + dt/2, k2)
        + function(tCurr + dt, k3)
    )

    # Check if skydiver is on the ground
    new_data = isSkydiverOnGround(new_data)

    return new_data

def plotEuler():
    times = np.linspace(0, totalTime, iterations)
    deltaTime = times[1] - times[0]

    data = np.zeros((2,iterations), dtype=float)

    # Initial values
    data[0,0] = h0
    data[1,0] = v0

    for i in range(1, iterations):
        data[:,i] = EulerStep(data[:,i-1], times[i-1], deltaTime)

    plt.figure()
    plt.grid()
    plt.title(f"Eulerio metodas(Aukstis) \ deltaT= {np.round(deltaTime, 5)}")
    plt.plot(times, data[0,:], label="Aukštis")

    plt.figure()
    plt.grid()
    plt.title(f"Eulerio metodas(Greitis) \ deltaT= {np.round(deltaTime, 5)}")
    plt.plot(times, data[1,:], label="Greitis")

def plotRK4():
    times = np.linspace(0, totalTime, iterations)
    deltaTime = times[1] - times[0]

    data = np.zeros((2,iterations), dtype=float)

    # Initial values
    data[0,0] = h0
    data[1,0] = v0

    for i in range(1, iterations):
        data[:,i] = RK4(data[:,i-1], times[i-1], deltaTime)

    plt.figure()
    plt.grid()
    plt.title(f"RK4(Aukstis) \ deltaT= {np.round(deltaTime, 5)}")
    plt.plot(times, data[0,:], label="Aukštis")

    plt.figure()
    plt.grid()
    plt.title(f"RK4(Greitis) \ deltaT= {np.round(deltaTime, 5)}")
    plt.plot(times, data[1,:], label="Greitis")

def plotAnalytical():
    times = np.linspace(0, totalTime, iterations)
    deltaTime = times[1] - times[0]

    data = np.zeros((2,iterations), dtype=float)

    # Initial values
    data[0,0] = h0
    data[1,0] = v0

    scipy_results = integrate.solve_ivp(fun = function, y0=(h0, v0), t_span=(0, totalTime), t_eval=times)
    heights = scipy_results.y[0]
    velocities = scipy_results.y[1]

    # Clean up velocities and heights when parachuter reached the ground
    for i in range(len(heights)):
        if heights[i] <= 0:
            heights[i] = 0
            velocities[i] = 0

    plt.figure()
    plt.grid()
    plt.title(f"Analitinis(Aukstis) \ deltaT= {np.round(deltaTime, 5)}")
    plt.plot(times, heights, label="Aukštis")

    plt.figure()
    plt.grid()
    plt.title(f"Analitinis(Greitis) \ deltaT= {np.round(deltaTime, 5)}")
    plt.plot(times, velocities, label="Greitis")

plotEuler()
plotRK4()
plotAnalytical()
plt.show()